Recomandari de filme folosind DVS (Descompunerea in valori singulare)

Studenti: Sava Catalin-Andrei,
          Istrate Angel-Andrei, Grupa: 322AC

Citim fisierele "ratings.dat" si "filme.dat" in doua matrici separate. Acestea contin date despre filme (ID, titlu si categorie) si despre spectatori (ID, ID-ul filmului vizionat, rating si timp)

In [32]:
import numpy as np
import pandas as pd

data = pd.io.parsers.read_csv('ratings.dat', 
    names=['user_id', 'film_id', 'rating', 'timp'],
    engine='python', delimiter='::',encoding='latin-1')
film_data = pd.io.parsers.read_csv('filme.dat',
    names=['film_id', 'titlu', 'gen'],
    engine='python', delimiter='::',encoding='latin-1')

Cream o matricea initiala cu filmele pe linii si userii pe coloane

In [33]:
ratings_mat = np.ndarray(
    shape=(np.max(data.film_id.values), np.max(data.user_id.values)),
    dtype=np.uint8)
ratings_mat[data.film_id.values-1, data.user_id.values-1] = data.rating.values

Normalizam matricea ratings_mat prin scaderea elementelor de pe fiecare linie cu media liniei

In [34]:
normalised_mat = ratings_mat - np.asarray([(np.mean(ratings_mat, 1))]).T

Punem in aplicare DVS

In [35]:
U, S, V = np.linalg.svd(normalised_mat.T)

Definim functia de Cosine Similarity

In [36]:
def top_cosine_similarity(data, film_id, top_n=10): #Avem nevoie de 3 parametrii: matricea, ID-ul filmului dorit si numarul de recomandari
    index = film_id - 1 #ID-ul filmelor incepe de la 1
    film_row = data[index, :] #Luam randul filmului dorit si toate coloanele aferente (stocam toate rating-urile filmului)
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data)) #Calculeaza magnitudinea (adunarea patratelor fiecarui element sub radical)
    similarity = np.dot(film_row, data.T) / (magnitude[index] * magnitude) #Aplicam Cosine Similarity
    sort_indexes = np.argsort(-similarity) #Sorteaza rezultatele descrescator
    return sort_indexes[:top_n] #Returneaza primele n recomandari

Folosim aceasta functie ajutatoare pentru a printa rezultatele

In [37]:
def print_similar_movies(film_data, film_id, top_indexes):
    print('Recommendations for {0}: \n'.format(film_data[film_data.film_id == film_id].titlu.values[0]))
    for id in top_indexes + 1:
        print(film_data[film_data.film_id == id].titlu.values[0])

Setam variabilele

In [38]:
k = 50 #Index pentru date reprezentative
film_id = 1 #ID-ul filmului pentru care vrem recomandari
top_n = 10 #Primele n recomandari

Cream o matrice de date reprezentative pentru a o transmite functiei top_cosine_similarity

In [39]:
sliced = V.T[:, :k] #Toate liniile si pana la coloana k
indexes = top_cosine_similarity(sliced, film_id, top_n) 
print_similar_movies(film_data, film_id, indexes)

Recommendations for Toy Story (1995): 

Toy Story (1995)
Toy Story 2 (1999)
Babe (1995)
Bug's Life, A (1998)
Pleasantville (1998)
Babe: Pig in the City (1998)
Aladdin (1992)
Stuart Little (1999)
Secret Garden, The (1993)
Tarzan (1999)


C:\Users\Sava\AppData\Local\Temp\ipykernel_1596\1224720441.py:5: RuntimeWarning: invalid value encountered in divide
  similarity = np.dot(film_row, data.T) / (magnitude[index] * magnitude) #Aplicam Cosine Similarity
